#### Spark

In [1]:
import pyspark 
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("practica_ibd").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/11 21:02:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#spark.stop()

In [ ]:
# Lee los archivos JSON en un DataFrame
archivos_json = spark.read.option("multiline", "true").json("json")

# Selecciona el campo 'author_name'
campos_seleccionados = archivos_json.select(
    archivos_json['author_name'].getItem(0).alias('author_name'),
    archivos_json['first_publish_year'].alias('publish_year'),
    archivos_json['title'].alias('title'),
    archivos_json['type'].alias('type'),
    archivos_json['isbn'].alias('isbn'),
    archivos_json['ebook_access'].alias('ebook_access')
    # Añadir los campos que haga falta
)

# Muestra el DataFrame resultante
campos_seleccionados.show()

#### Elasticsearch

In [ ]:
pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import json

# Función para convertir un DataFrame de Spark a un formato compatible con Elasticsearch
def df_to_es(df):
    for row in df.toJSON().collect():
        yield json.loads(row)

# Conectar con Elasticsearch
client = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])  # Reemplaza localhost con la dirección de tu Elasticsearch si es necesario

# Definir un DataFrame de Spark con las columnas seleccionadas
# Suponiendo que tienes un DataFrame llamado 'campos_seleccionados' correctamente definido

# Indexar documentos en Elasticsearch utilizando Spark
try:
    helpers.bulk(client, df_to_es(campos_seleccionados), index='my-index')
    print("Documentos indexados correctamente.")
except Exception as e:
    print("Error al indexar documentos:", e)

# Realizar una consulta para recuperar algunos documentos
try:
    res = client.search(index='my-index', size=5)  
    for hit in res['hits']['hits']:
        print(hit['_source'])
except Exception as e:
    print("Error al realizar la consulta:", e)

# Cierra la sesión de Spark
#spark.stop()